In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
from sklearn.utils import shuffle
import jieba
import re
import time
import collections
import os

In [4]:
question = open("question.csv",encoding='utf-8')

answer = open("answer.csv",encoding='utf-8')

all_train_id = pd.read_table("train_candidates.txt",sep=',',dtype=str,nrows = None)

In [5]:
# # txt去重，太慢了
# train_set = []
# for line in train_id:
#     que_to_ans = line.strip('\n').split(",")[:2]
#     if que_to_ans not in train_set:
#         print(que_to_ans)
#         train_set.append(que_to_ans)
# train_set

del all_train_id["neg_ans_id"]
all_train_id.drop_duplicates(inplace=True)
all_train_id.reset_index(drop=True, inplace=True)
train_id = all_train_id[:2000]

In [6]:
# 简单处理一下question/answer每句话
def sentence_precessing(text):
    
    text = text.lower().strip()
    
    text = re.sub(r'["cm"]+', "厘米", text)
    text = re.sub(r"([。])", r".", text)
    text = re.sub(r"[a-zA-Z]", "",text)
    text = re.sub(r"([？?!！，¿])", r",", text)
    text = re.sub(r"([‘’“”\"\"''])", r"", text)
    text = re.sub(r"[-()（）\#/@;:<>{}`+=~|]", "", text)
    text = re.sub(r'[" "]+', "", text)
    
    return text

In [7]:
question_dict = {}
question.readline()
for line in question:
    _line = line.strip('\n').split(',')
    question_dict[_line[0]] = " ".join(jieba.cut(sentence_precessing(_line[1])))

answer_dict = {}
answer.readline()
for line in answer:
    _line = line.strip('\n').split(',')
    answer_dict[_line[0]] = " ".join(jieba.cut(sentence_precessing(_line[2])))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\leo\AppData\Local\Temp\jieba.cache
Loading model cost 0.940 seconds.
Prefix dict has been built succesfully.


In [8]:
train_questions = []
train_answers = []
for question,answer in zip(train_id["question_id"],train_id["pos_ans_id"]):
    train_questions.append(question_dict[question])
    train_answers.append(answer_dict[answer])

In [9]:
def build_dataset(words, n_words, atleast=10):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [10]:
concat_from = ' '.join(train_questions).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 4757
Most common words [(',', 5694), ('了', 1311), ('的', 1259), ('.', 1064), ('我', 962), ('是', 783)]
Sample data [170, 41, 25, 0, 42, 141, 11, 4, 215, 14] ['不是', '说', '做', 'PAD', '宝宝', '不好', '吗', ',', '那', '怀孕']
filtered vocab size: 803
% of vocab used: 16.88%


In [11]:
concat_to = ' '.join(train_answers).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 10294
Most common words [(',', 10837), ('的', 6329), ('.', 4774), ('、', 2278), ('是', 2057), ('治疗', 1523)]
Sample data [254, 197, 0, 298, 17, 8, 274, 5, 352, 26] ['超', '属于', 'PAD', '经常', '检查', '是', '不好', '的', '而且', '也']
filtered vocab size: 1413
% of vocab used: 13.73%


In [12]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [13]:
for i in range(len(train_answers)):
    train_answers[i] += ' EOS'

In [14]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [15]:
X = str_idx(train_questions, dictionary_from)
Y = str_idx(train_answers, dictionary_to)
# X_test = str_idx(question_test, dictionary_from)
# Y_test = str_idx(answer_test, dictionary_from)

In [16]:
def pad_sentence_batch(sentence_batch, pad_int, maxlen):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = maxlen
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(maxlen)
    return padded_seqs, seq_lens

In [17]:
# embedded_size = 256
# learning_rate = 1e-3
# batch_size = 32
# epoch = 100
embedded_size = 64
learning_rate = 1e-3
batch_size = 32
epoch = 100

In [18]:
maxlen_question = max([len(x) for x in X])
maxlen_answer = max([len(y) for y in Y])

In [19]:
def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))

    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    
    outputs = gamma * normalized + beta
    return outputs


def multihead_attn(queries, keys, q_masks, k_masks, future_binding, num_units, num_heads):
    
    T_q = tf.shape(queries)[1]                                      
    T_k = tf.shape(keys)[1]                  

    Q = tf.layers.dense(queries, num_units, name='Q')                              
    K_V = tf.layers.dense(keys, 2*num_units, name='K_V')    
    K, V = tf.split(K_V, 2, -1)        

    Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0)                         
    K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0)                    
    V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0)                      

    align = tf.matmul(Q_, tf.transpose(K_, [0,2,1]))                      
    align = align / np.sqrt(K_.get_shape().as_list()[-1])                 

    paddings = tf.fill(tf.shape(align), float('-inf'))                   

    key_masks = k_masks                                                 
    key_masks = tf.tile(key_masks, [num_heads, 1])                       
    key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, T_q, 1])            
    align = tf.where(tf.equal(key_masks, 0), paddings, align)       

    if future_binding:
        lower_tri = tf.ones([T_q, T_k])                                          
        lower_tri = tf.linalg.LinearOperatorLowerTriangular(lower_tri).to_dense()  
        masks = tf.tile(tf.expand_dims(lower_tri,0), [tf.shape(align)[0], 1, 1]) 
        align = tf.where(tf.equal(masks, 0), paddings, align)                      
    
    align = tf.nn.softmax(align)                                            
    query_masks = tf.to_float(q_masks)                                             
    query_masks = tf.tile(query_masks, [num_heads, 1])                             
    query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, T_k])            
    align *= query_masks                                                           
          
    outputs = tf.matmul(align, V_)                                                 
    outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2)             
    outputs += queries                                                             
    outputs = layer_norm(outputs)                                                 
    return outputs


def pointwise_feedforward(inputs, hidden_units, activation=None):
    outputs = tf.layers.dense(inputs, 4*hidden_units, activation=activation)
    outputs = tf.layers.dense(outputs, hidden_units, activation=None)
    outputs += inputs
    outputs = layer_norm(outputs)
    return outputs


def learned_position_encoding(inputs, mask, embed_dim):
    T = tf.shape(inputs)[1]
    outputs = tf.range(tf.shape(inputs)[1])                # (T_q)
    outputs = tf.expand_dims(outputs, 0)                   # (1, T_q)
    outputs = tf.tile(outputs, [tf.shape(inputs)[0], 1])   # (N, T_q)
    outputs = embed_seq(outputs, T, embed_dim, zero_pad=False, scale=False)
    return tf.expand_dims(tf.to_float(mask), -1) * outputs


def sinusoidal_position_encoding(inputs, mask, repr_dim):
    T = tf.shape(inputs)[1]
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1]) * tf.expand_dims(tf.to_float(mask), -1)


def label_smoothing(inputs, epsilon=0.1):
    C = inputs.get_shape().as_list()[-1]
    return ((1 - epsilon) * inputs) + (epsilon / C)


### tensorflow2.0可使用loss_fun代替tf.contrib.seq2seq.sequence_loss

In [20]:
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
#                                                            reduction='none')

# def loss_fun(y_ture, y_pred):
#     mask = tf.math.logical_not(tf.math.equal(y_ture, 0))  # 为0掩码标1
#     loss_ = loss_object(y_ture, y_pred)
    
#     mask = tf.cast(mask, dtype=loss_.dtype)
#     loss_ *= mask
#     return tf.reduce_mean(loss_)

In [21]:
class Chatbot:
    def __init__(self, size_layer, embedded_size, from_dict_size, to_dict_size, learning_rate,
                 num_blocks = 2,
                 num_heads = 8,
                 min_freq = 50):
        self.X = tf.placeholder(tf.int32,[None,None])
        self.Y = tf.placeholder(tf.int32,[None,None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        def forward(x, y):
            encoder_embedded = tf.nn.embedding_lookup(encoder_embedding, x)
            en_masks = tf.sign(x)
            encoder_embedded += sinusoidal_position_encoding(x, en_masks, embedded_size)
        
            for i in range(num_blocks):
                with tf.variable_scope('encoder_self_attn_%d'%i,reuse=tf.AUTO_REUSE):
                    encoder_embedded = multihead_attn(queries = encoder_embedded,
                                             keys = encoder_embedded,
                                             q_masks = en_masks,
                                             k_masks = en_masks,
                                             future_binding = False,
                                             num_units = size_layer,
                                             num_heads = num_heads)

                with tf.variable_scope('encoder_feedforward_%d'%i,reuse=tf.AUTO_REUSE):
                    encoder_embedded = pointwise_feedforward(encoder_embedded,
                                                    embedded_size,
                                                    activation = tf.nn.relu)
            
            decoder_embedded = tf.nn.embedding_lookup(decoder_embedding, y)
            de_masks = tf.sign(y)
            decoder_embedded += sinusoidal_position_encoding(y, de_masks, embedded_size)
            
            for i in range(num_blocks):
                with tf.variable_scope('decoder_self_attn_%d'%i,reuse=tf.AUTO_REUSE):
                    decoder_embedded = multihead_attn(queries = decoder_embedded,
                                         keys = decoder_embedded,
                                         q_masks = de_masks,
                                         k_masks = de_masks,
                                         future_binding = True,
                                         num_units = size_layer,
                                         num_heads = num_heads)
                
                with tf.variable_scope('decoder_attn_%d'%i,reuse=tf.AUTO_REUSE):
                    decoder_embedded = multihead_attn(queries = decoder_embedded,
                                         keys = encoder_embedded,
                                         q_masks = de_masks,
                                         k_masks = en_masks,
                                         future_binding = False,
                                         num_units = size_layer,
                                         num_heads = num_heads)
                
                with tf.variable_scope('decoder_feedforward_%d'%i,reuse=tf.AUTO_REUSE):
                    decoder_embedded = pointwise_feedforward(decoder_embedded,
                                                    embedded_size,
                                            activation = tf.nn.relu)
            
            return tf.layers.dense(decoder_embedded, to_dict_size, reuse=tf.AUTO_REUSE)
        
        self.training_logits = forward(self.X, decoder_input)
        
        def cond(i, y, temp):
            return i < 2 * tf.reduce_max(self.X_seq_len)
        
        def body(i, y, temp):
            logits = forward(self.X, y)
            ids = tf.argmax(logits, -1)[:, i]
            ids = tf.expand_dims(ids, -1)
            temp = tf.concat([temp[:, 1:], ids], -1)
            y = tf.concat([temp[:, -(i+1):], temp[:, :-(i+1)]], -1)
            y = tf.reshape(y, [tf.shape(temp)[0], 2 * tf.reduce_max(self.X_seq_len)])
            i += 1
            return i, y, temp
        
        target = tf.fill([batch_size, 2 * tf.reduce_max(self.X_seq_len)], GO)
        target = tf.cast(target, tf.int64)
        self.target = target
        
        _, self.predicting_ids, _ = tf.while_loop(cond, body, 
                                                  [tf.constant(0), target, target])
        self.logits = forward(self.X, self.Y)
        self.k = tf.placeholder(dtype = tf.int32)
        p = tf.nn.softmax(self.logits)
        self.topk_logprobs, self.topk_ids = tf.nn.top_k(tf.log(p), self.k)
        
        # masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        masks = tf.sequence_mask(self.Y_seq_len, maxlen_answer, dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,targets = self.Y,weights = masks)
        # tensorflow2.0可代替
#         self.cost = loss_fun(self.Y, self.training_logits)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
# model = Chatbot(embedded_size, embedded_size, len(dictionary_from), 
#                 len(dictionary_to), learning_rate)
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()
# ckpt = tf.train.latest_checkpoint('models/')
# saver.restore(sess, ckpt)

tf.reset_default_graph()
sess = tf.Session()
model = Chatbot(embedded_size, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate)
sess.run(tf.global_variables_initializer())

In [23]:
import time
saver = tf.train.Saver()
for i in range(epoch):
    start = time.time()
    total_loss, total_accuracy = 0, 0
    for k in range(0, len(train_questions), batch_size):
        index = min(k+batch_size, len(train_questions))
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD, maxlen_question)
        batch_y, seq_y = pad_sentence_batch(Y[k: index], PAD, maxlen_answer)
        accuracy,loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x, model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
    total_loss /= (len(train_questions) / batch_size)
    total_accuracy /= (len(train_questions) / batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))
    saver.save(sess, 'models/transfermer_model')
    print('time in 1 epoch:{} secs\n'.format(time.time()-start))
    
    if i % 5 == 0:
        start = time.time()
        total_loss, total_accuracy = 0, 0
        
        


KeyboardInterrupt: 

In [ ]:
batch_x, seq_x = pad_sentence_batch(X[20:40], PAD, maxlen_question)
batch_y, seq_y = pad_sentence_batch(Y[20:40], PAD, maxlen_answer)
predicted, accuracy,loss, _ = sess.run([model.predicting_ids, model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,model.Y:batch_y})
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

In [ ]:
maxlen_question,maxlen_answer

In [ ]:
sess.close()

In [ ]:
# saver = tf.train.Saver()
# saver.save(sess, 'my_test_model')

In [ ]:
# for i in range(len(batch_x)):
#     print('row %d'%(i+1))
#     print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
#     print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
#     print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')